# Brewery Recommender: A bunch of tests

In [65]:
def freq_counter(int_list):
    count = 1
    num_dict = {}
    sort_list = sorted(int_list)
    for i in range(1, len(sort_list)):
        if sort_list[i] == sort_list[i-1]:
            count += 1
            if i == len(sort_list) - 1:
                num_dict[str(sort_list[i])] = count
        else:
            num_dict[str(sort_list[i-1])] = count
            count = 1
    smallest = min(num_dict.values())
    for key, value in num_dict.items():
        if value == smallest:
            return int(key)

In [67]:
freq_counter([1,1, 1,22,22, 22, 8,8, 3, 4, 3,4,3,4])

8

In [456]:
# importing libraries
import pandas as pd
import sys
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt

from tqdm import tqdm
import numpy as np
import requests
from bs4 import BeautifulSoup

In [457]:
# reading in data
df = pd.read_csv('../data/beer_reviews_col_drop.csv')
df.head()

,brewery_name,review_overall,review_profilename,beer_name
0,Caldera Brewing Company,3.0,oline73,Caldera Ginger Beer
1,Caldera Brewing Company,3.5,Reidrover,Caldera Ginger Beer
2,Caldera Brewing Company,3.0,alpinebryant,Caldera Ginger Beer
3,Caldera Brewing Company,4.0,LordAdmNelson,Caldera Ginger Beer
4,Caldera Brewing Company,4.5,augustgarage,Caldera Ginger Beer


In [8]:
list(df['beer_name'].value_counts()[:10].index)

['90 Minute IPA',
 'Old Rasputin Russian Imperial Stout',
 'Sierra Nevada Celebration Ale',
 'Two Hearted Ale',
 'Stone Ruination IPA',
 'Arrogant Bastard Ale',
 'Sierra Nevada Pale Ale',
 'Stone IPA (India Pale Ale)',
 'Pliny The Elder',
 'Founders Breakfast Stout']

In [6]:
from sqlalchemy import create_engine

engine = create_engine('sqlite://', echo=False)

# df = pd.read_csv('../data/beer_reviews_col_drop.csv').head()

df.to_sql('beer_test', con=engine)

KeyboardInterrupt: 

In [72]:
df.columns

Index(['brewery_name', 'review_overall', 'review_profilename', 'beer_name'], dtype='object')

In [95]:
engine.execute("\
               SELECT COUNT(beer_name), beer_name\ 
               FROM beer_test\ 
               GROUP BY beer_name\ 
               ORDER BY COUNT(beer_name) DESC LIMIT 10\
               ").fetchall()

[(3288, '90 Minute IPA'),
 (3126, 'India Pale Ale'),
 (3097, 'Old Rasputin Russian Imperial Stout'),
 (2986, 'Sierra Nevada Celebration Ale'),
 (2726, 'Two Hearted Ale'),
 (2702, 'Stone Ruination IPA'),
 (2697, 'Arrogant Bastard Ale'),
 (2577, 'Sierra Nevada Pale Ale'),
 (2573, 'Stone IPA (India Pale Ale)'),
 (2527, 'Pliny The Elder')]

In [10]:
# using a pivot table to create my recommender model
pivot = df.pivot_table(index='beer_name', columns='review_profilename', 
                       values=['review_overall'])
pivot_sparse = sparse.csr_matrix(pivot.fillna(0))
recommender = pairwise_distances(pivot_sparse, metric='cosine')
recommender_df = pd.DataFrame(recommender, index=pivot.index, columns=pivot.index)

In [29]:
# creating a smaller dataframe because in my flask demo the user will only be rating the top 10 beers
recommender_demo = recommender_df[list(df['beer_name'].value_counts()[:10].index)]

In [31]:
recommender_demo.info() # checking the size of my new dataframe

<class 'pandas.core.frame.DataFrame'>
Index: 37894 entries, "100" to Ω-naught (Omeganaught)
Data columns (total 10 columns):
90 Minute IPA                          37894 non-null float64
Old Rasputin Russian Imperial Stout    37894 non-null float64
Sierra Nevada Celebration Ale          37894 non-null float64
Two Hearted Ale                        37894 non-null float64
Stone Ruination IPA                    37894 non-null float64
Arrogant Bastard Ale                   37894 non-null float64
Sierra Nevada Pale Ale                 37894 non-null float64
Stone IPA (India Pale Ale)             37894 non-null float64
Pliny The Elder                        37894 non-null float64
Founders Breakfast Stout               37894 non-null float64
dtypes: float64(10)
memory usage: 4.4+ MB


In [33]:
# making sure everything looks okay
recommender_demo.head()

beer_name,90 Minute IPA,Old Rasputin Russian Imperial Stout,Sierra Nevada Celebration Ale,Two Hearted Ale,Stone Ruination IPA,Arrogant Bastard Ale,Sierra Nevada Pale Ale,Stone IPA (India Pale Ale),Pliny The Elder,Founders Breakfast Stout
beer_name,,,,,,,,,,
"""100""",0.977350,0.974868,0.983247,0.973980,0.978240,0.974605,0.991669,0.984429,0.957744,0.974193
"""12"" Belgian Golden Strong Ale",0.987316,0.975547,0.989093,0.985759,0.986893,0.973231,0.972009,0.985228,0.984462,0.973885
"""33"" Export",0.978069,0.978786,0.972012,0.979974,0.975922,0.976778,0.970423,0.975877,0.987313,0.977345
"""4"" Horse Oatmeal Stout",0.989555,0.975547,0.970396,0.968353,0.977063,0.974904,0.968716,0.968814,1.000000,0.985311
"""400"" Ale",0.943148,0.950567,0.927963,0.940541,0.926437,0.944057,0.946979,0.933717,0.972655,0.956835


In [70]:
# saving the recommender_demo dataframe to a csv
recommender_demo.to_csv('../data/recommender_demo.csv')

In [458]:
recommender_df = pd.read_csv('../data/recommender_demo.csv')
recommender_df.head()

,beer_name,90 Minute IPA,Old Rasputin Russian Imperial Stout,Sierra Nevada Celebration Ale,Two Hearted Ale,Stone Ruination IPA,Arrogant Bastard Ale,Sierra Nevada Pale Ale,Stone IPA (India Pale Ale),Pliny The Elder,Founders Breakfast Stout
0,"""100""",0.977350,0.974868,0.983247,0.973980,0.978240,0.974605,0.991669,0.984429,0.957744,0.974193
1,"""12"" Belgian Golden Strong Ale",0.987316,0.975547,0.989093,0.985759,0.986893,0.973231,0.972009,0.985228,0.984462,0.973885
2,"""33"" Export",0.978069,0.978786,0.972012,0.979974,0.975922,0.976778,0.970423,0.975877,0.987313,0.977345
3,"""4"" Horse Oatmeal Stout",0.989555,0.975547,0.970396,0.968353,0.977063,0.974904,0.968716,0.968814,1.000000,0.985311
4,"""400"" Ale",0.943148,0.950567,0.927963,0.940541,0.926437,0.944057,0.946979,0.933717,0.972655,0.956835


In [459]:
recommender_df.set_index('beer_name', inplace=True)
recommender_df.head()

,90 Minute IPA,Old Rasputin Russian Imperial Stout,Sierra Nevada Celebration Ale,Two Hearted Ale,Stone Ruination IPA,Arrogant Bastard Ale,Sierra Nevada Pale Ale,Stone IPA (India Pale Ale),Pliny The Elder,Founders Breakfast Stout
beer_name,,,,,,,,,,
"""100""",0.977350,0.974868,0.983247,0.973980,0.978240,0.974605,0.991669,0.984429,0.957744,0.974193
"""12"" Belgian Golden Strong Ale",0.987316,0.975547,0.989093,0.985759,0.986893,0.973231,0.972009,0.985228,0.984462,0.973885
"""33"" Export",0.978069,0.978786,0.972012,0.979974,0.975922,0.976778,0.970423,0.975877,0.987313,0.977345
"""4"" Horse Oatmeal Stout",0.989555,0.975547,0.970396,0.968353,0.977063,0.974904,0.968716,0.968814,1.000000,0.985311
"""400"" Ale",0.943148,0.950567,0.927963,0.940541,0.926437,0.944057,0.946979,0.933717,0.972655,0.956835


In [460]:
recommender_df['90 Minute IPA'].sort_values()[1:10]

beer_name
60 Minute IPA                                 0.467290
Stone Ruination IPA                           0.478857
Stone IPA (India Pale Ale)                    0.482735
HopDevil Ale                                  0.492149
Storm King Stout                              0.495548
Old Rasputin Russian Imperial Stout           0.498012
Sierra Nevada Bigfoot Barleywine Style Ale    0.500069
Arrogant Bastard Ale                          0.502324
Sierra Nevada Celebration Ale                 0.507578
Name: 90 Minute IPA, dtype: float64

In [461]:
%%time
# making sure the recommender works for 1 beer first
recommender_df['Old Rasputin Russian Imperial Stout'].sort_values()[1:10]

CPU times: user 9.13 ms, sys: 1.08 ms, total: 10.2 ms
Wall time: 9.1 ms


beer_name
Stone Imperial Russian Stout                  0.491970
Storm King Stout                              0.493446
Stone Ruination IPA                           0.497395
Sierra Nevada Bigfoot Barleywine Style Ale    0.497664
90 Minute IPA                                 0.498012
Brooklyn Black Chocolate Stout                0.505900
Sierra Nevada Celebration Ale                 0.510606
St. Bernardus Abt 12                          0.512458
Ayinger Celebrator Doppelbock                 0.514944
Name: Old Rasputin Russian Imperial Stout, dtype: float64

In [535]:
np.random.seed(60)
ratings = {}
for i in recommender_df.columns:
    ratings[i] = np.random.uniform(low=1, high=5)

In [558]:
ratings

{'90 Minute IPA': 2.203493320186475,
 'Old Rasputin Russian Imperial Stout': 1.747783265367512,
 'Sierra Nevada Celebration Ale': 2.292730707872878,
 'Two Hearted Ale': 3.6629982811532362,
 'Stone Ruination IPA': 3.2678832038768535,
 'Arrogant Bastard Ale': 2.5930158355950588,
 'Sierra Nevada Pale Ale': 2.5176596620983864,
 'Stone IPA (India Pale Ale)': 1.0423261435145719,
 'Pliny The Elder': 1.6814624039748485,
 'Founders Breakfast Stout': 1.4935734827204161}

In [559]:
recs = []
rating_scale = []
for key, value in ratings.items():
    rec = {}
#     if value >= 3.5:
    rec[key] = recommender_df[key].sort_values()[1:11]
    recs.append(rec)
    rating_scale.append(value)

In [567]:
count = 0
scaler_count = 0
scaler = rating_scale[scaler_count]
beer_checklist = []

for i in recs:
    for beer, scores in i.items():
        for recom, score in scores.items():
            count += 1
            if count % 11 == 0:
                scaler_count += 1
                try:
                    scaler = rating_scale[scaler_count]
                except:
                    continue
            if scaler > 3:
                scaled_value = 1 + ((scaler - 3) / scaler)
            else:
                scaled_value = 1 - ((3 - scaler) / 3)

            new_score = score / scaled_value
            if new_score >= 1:
                new_score = 1
            beer_checklist.append((recom, new_score))

In [574]:
def brewery_recommender(user):
    # creating a list of the 20 most similar beers if the user rated these beers equal to or above 3.5 (0-5 scale)
    recs = []
    rating_scale = []

    for key, value in user.items():
        rec = {}
#         if value >= 3.5:
        rec[key] = recommender_df[key].sort_values()[1:11]
        recs.append(rec)
        rating_scale.append(value)

    count = 0
    scaler_count = 0
    scaler = rating_scale[scaler_count]
    beer_checklist = []

    for i in recs:
        for beer, scores in i.items():
            for recom, score in scores.items():
                count += 1
                if count % 11 == 0:
                    scaler_count += 1
                    try:
                        scaler = rating_scale[scaler_count]
                    except:
                        continue
                if scaler > 3:
                    scaled_value = 1 + ((scaler - 3) / scaler)
                else:
                    scaled_value = 1 - ((3 - scaler) / 3)

                new_score = score / scaled_value
                if new_score >= 1:
                    new_score = 1
                beer_checklist.append((recom, new_score))

    
    # creating a list of breweries and the beer scores associated with that brewery to find the most 
    # similar brewery based on the mean similarity score
    breweries = []
    current_beer = ''
    for beer in beer_checklist:
        for index, brewery in df[df['beer_name'] == beer[0]]['brewery_name'].items():
            if current_beer != beer[0]:
                current_beer = beer[0] 
                brewery_score = (brewery, beer[1], beer[0])
                breweries.append(brewery_score) 
                
    # sorting the breweries alphabetically to make it easier to loop through
    current_brewery = ''
    scores = []
    brewery_dict = {}
    beer_dict = {}
    beers_dict = {}
    breweries_sort = sorted(breweries)
    for i in range(len(breweries_sort)):
        if breweries_sort[i][0] == current_brewery:
            current_beer = breweries_sort[i][2]
            beer_dict[current_beer] = breweries_sort[i][1]      
            scores.append(breweries_sort[i][1])
        else:
            if current_brewery:
                brewery_dict[current_brewery] = scores
                beers_dict[current_brewery] = beer_dict
                beer_dict = {}
            current_brewery = breweries_sort[i][0]
            current_beer = breweries_sort[i][2]

            beer_dict[current_beer] = breweries_sort[i][1]     
            scores = []
            scores.append(breweries_sort[i][1])
    return beers_dict

# creating a random user that will rate num_beers on a 0-5 uniform distribution scale
# def random_beer_sample(num_beers):
#     beers = df['beer_name'].sample(n=num_beers, random_state=42) 
#     np.random.seed(6)
#     ratings = []
#     user_ratings = {}
#     for i in beers:
#         user_ratings[i] = np.random.uniform(high=5.0)
#     return user_ratings

# sorting the breweries mean similarity scores and counting the number of similar beers at that brewery 
# similar beers are beers with similarity scores less than 0.5








def sorting_brewery_scores(rand_user):
    brewery_score = {}
    scores = []
    for brewery, beer_scores in brewery_recommender(rand_user).items():
        scores_brew = []
        brewery_score[brewery] = scores_brew
        for beer, score in beer_scores.items():
            scores_brew.append(score)
        scores.append(scores_brew)

    sim_beers_all = []
    breweries = []
    scores = []
    best_breweries = {}

    for brewery, score in brewery_score.items():
        sim_beers = 0
        for j in score:
            if j < 0.5:
                sim_beers += 1
        breweries.append(brewery)
        sim_beers_all.append(sim_beers)
        scores.append(np.mean(score))

    best_breweries['brewery'] = breweries
    best_breweries['similar_beers'] = sim_beers_all
    best_breweries['score'] = scores
                
#         if 2 <= sim_beers:
#         best_breweries[brewery] = (np.mean(score), 'number of similar beers: ', sim_beers)
#         else:
#             continue
        
#     sorted_best_breweries = sorted(best_breweries.items(), key = 
#                  lambda kv:(kv[1], kv[0]))
#     if len(sorted_best_breweries) > 3:
#         sorted_best_breweries = sorted_best_breweries[:3]
#     if sorted_best_breweries[0][1][0] > 0.5:
#         sorted_best_breweries = sorted_best_breweries[0][0]
#     else:
#         sorted_best_breweries = sorted_best_breweries
    
    return best_breweries

def data_manipulation(rand_user):
    recs = sorting_brewery_scores(rand_user)
    data = pd.DataFrame(recs)
    data.set_index('brewery', inplace=True)
    new_recs = data.sort_values(['score']).loc[(data.sort_values(['score'])['similar_beers'] >= 1) & 
                                    (data.sort_values(['score'])['score'] < 0.5)]
    return new_recs


In [575]:
np.random.seed(60)
ratings = {}
for i in recommender_df.columns:
    ratings[i] = np.random.uniform(low=2.5, high=5)

recs = brewery_recommender(ratings)

In [576]:
recs

{'Anchor Brewing Company': {'Anchor Steam Beer': 0.4784188354632414,
  'Anchor Liberty Ale': 0.4879616373005337},
 'Ballast Point Brewing Company': {'Sculpin India Pale Ale': 0.5904210033069421},
 'Bear Republic Brewing Co.': {'Hop Rod Rye': 0.5847381869925506,
  'Racer 5 India Pale Ale': 0.5888943661052445},
 "Bell's Brewery, Inc.": {"Bell's Kalamazoo Stout": 0.38361861400824887,
  "Bell's Oberon Ale": 0.39654992366610653,
  "Bell's Expedition Stout": 0.5246125439207497,
  "Bell's Hopslam Ale": 0.49998003595351553,
  'Two Hearted Ale': 0.5154922752649316},
 'Boston Beer Company (Samuel Adams)': {'Samuel Adams Boston Lager': 0.427441397775485},
 'Brasserie Dieu Du Ciel': {'Péché Mortel (Imperial Stout Au Cafe)': 0.5632695219382423},
 'Brooklyn Brewery': {'Brooklyn Black Chocolate Stout': 0.5114641502594749},
 'Brouwerij St. Bernardus NV': {'St. Bernardus Abt 12': 0.5180941133565065},
 'Deschutes Brewery': {'The Abyss': 0.6050922871016279},
 'Dogfish Head Brewery': {'90 Minute IPA': 0.5

In [262]:
brewery_score = {}
scores = []
for brewery, beer_scores in recs.items():
    scores_brew = []
    brewery_score[brewery] = scores_brew
    for beer, score in beer_scores.items():
        scores_brew.append(score)
    scores.append(scores_brew)

sim_beers_all = []
breweries = []
scores = []
best_breweries = {}
best_breweries_beers = {}

for brewery, beer_scores in recs.items():
    beers = []
    for beer, score in beer_scores.items():
        if score < 0.5:
            beers.append(beer)
    best_breweries_beers[brewery] = beers

for brewery, score in brewery_score.items():
    sim_beers = 0
    for j in score:
        if j < 0.5:
            sim_beers += 1
    breweries.append(brewery)
    sim_beers_all.append(sim_beers)
    scores.append(np.mean(score))
        
best_breweries['brewery'] = breweries
best_breweries['similar_beers'] = sim_beers_all
best_breweries['score'] = scores

pd.DataFrame(best_breweries).sort_values('similar_beers', ascending=False)

,brewery,similar_beers,score
6,Stone Brewing Co.,4,0.484651
1,"Bell's Brewery, Inc.",3,0.484428
0,Bear Republic Brewing Co.,1,0.488232
2,Dogfish Head Brewery,1,0.478857
4,North Coast Brewing Co.,1,0.497395
3,Founders Brewing Company,0,0.506955
5,Sierra Nevada Brewing Co.,0,0.508790


In [303]:
beers = {}
for i in best_breweries['brewery']:
    beer_list = []
    check = 1
    for brewery, beer in best_breweries_beers.items():
        if i == brewery and len(beer) > 0:
            beer_list = beer
            check = 0
    if check == 0:
        beers[i] = beer_list

In [304]:
beers

{'Bear Republic Brewing Co.': ['Hop Rod Rye'],
 "Bell's Brewery, Inc.": ["Bell's Expedition Stout",
  "Bell's Hopslam Ale",
  "Bell's Kalamazoo Stout"],
 'Dogfish Head Brewery': ['90 Minute IPA'],
 'North Coast Brewing Co.': ['Old Rasputin Russian Imperial Stout'],
 'Stone Brewing Co.': ['Arrogant Bastard Ale',
  'Double Bastard Ale',
  'Stone Imperial Russian Stout',
  'Oaked Arrogant Bastard Ale']}

In [223]:
np.random.seed(60)
ratings = {}
for i in recommender_df.columns:
    ratings[i] = np.random.uniform(low=2.5, high=5)

data_manipulation(ratings)

,similar_beers,score
brewery,,
Dogfish Head Brewery,1,0.478857
"Bell's Brewery, Inc.",3,0.484428
Stone Brewing Co.,4,0.484651
Bear Republic Brewing Co.,1,0.488232
North Coast Brewing Co.,1,0.497395


In [583]:
def brewery_recommender(user):
    # creating a list of the 20 most similar beers if the user rated these beers equal to or above 3.5 (0-5 scale)
    recs = []
    rating_scale = []

    for key, value in user.items():
        rec = {}
#         if value >= 3.5:
        rec[key] = recommender_df[key].sort_values()[1:11]
        recs.append(rec)
        rating_scale.append(value)

    count = 0
    scaler_count = 0
    scaler = rating_scale[scaler_count]
    beer_checklist = []

    for i in recs:
        for beer, scores in i.items():
            for recom, score in scores.items():
                count += 1
                if count % 11 == 0:
                    scaler_count += 1
                    try:
                        scaler = rating_scale[scaler_count]
                    except:
                        continue
                if scaler > 3:
                    scaled_value = 1 + ((scaler - 3) / scaler)
                else:
                    scaled_value = 1 - ((3 - scaler) / 3)

                new_score = score / scaled_value
                if new_score >= 1:
                    new_score = 1
                beer_checklist.append((recom, new_score))
 
    
    # creating a list of breweries and the beer scores associated with that brewery to find the most 
    # similar brewery based on the mean similarity score
    breweries = []
    current_beer = ''
    for beer in beer_checklist:
        for index, brewery in df[df['beer_name'] == beer[0]]['brewery_name'].items():
            if current_beer != beer[0]:
                current_beer = beer[0] 
                brewery_score = (brewery, beer[1], beer[0])
                breweries.append(brewery_score) 
                
    # sorting the breweries alphabetically to make it easier to loop through
    current_brewery = ''
    scores = []
    brewery_dict = {}
    beer_dict = {}
    beers_dict = {}
    breweries_sort = sorted(breweries)
    for i in range(len(breweries_sort)):
        if breweries_sort[i][0] == current_brewery:
            current_beer = breweries_sort[i][2]
            beer_dict[current_beer] = breweries_sort[i][1]      
            scores.append(breweries_sort[i][1])
        else:
            if current_brewery:
                brewery_dict[current_brewery] = scores
                beers_dict[current_brewery] = beer_dict
                beer_dict = {}
            current_brewery = breweries_sort[i][0]
            current_beer = breweries_sort[i][2]

            beer_dict[current_beer] = breweries_sort[i][1]     
            scores = []
            scores.append(breweries_sort[i][1])
    return beers_dict

def sorting_brewery_scores(rand_user):
    brewery_score = {}
    scores = []
    for brewery, beer_scores in brewery_recommender(rand_user).items():
        scores_brew = []
        brewery_score[brewery] = scores_brew
        for beer, score in beer_scores.items():
            scores_brew.append(score)
        scores.append(scores_brew)

    sim_beers_all = []
    breweries = []
    scores = []
    best_breweries = {}
    best_breweries_beers = {}

    for brewery, beer_scores in brewery_recommender(rand_user).items():
        beers = []
        for beer, score in beer_scores.items():
            if score < 0.5:
                beers.append(beer)
        best_breweries_beers[brewery] = beers
        
    for brewery, score in brewery_score.items():
        sim_beers = 0
        for j in score:
            if j < 0.5:
                sim_beers += 1
        breweries.append(brewery)
        sim_beers_all.append(sim_beers)
        scores.append(np.mean(score))

    best_breweries['brewery'] = breweries
    best_breweries['similar_beers'] = sim_beers_all
    best_breweries['score'] = scores
    
    return best_breweries, best_breweries_beers

def data_manipulation(rand_user):
    recs = sorting_brewery_scores(rand_user)
    data = pd.DataFrame(recs[0])
    beers_best = recs[1]
#     data.set_index('brewery', inplace=True)
    new_recs = data.sort_values(['score']).loc[(data.sort_values(['score'])['similar_beers'] >= 2) & 
                                    (data.sort_values(['score'])['score'] < 1)]
    
    new_recs = pd.DataFrame(new_recs).sort_values('similar_beers', ascending=False)
#     new_recs = new_recs.index
    beers = {}
    for i in new_recs['brewery']:
        beer_list = []
        check = 1
        for brewery, beer in beers_best.items():
            if i == brewery and len(beer) > 0:
                beer_list = beer
                check = 0
        if check == 0:
            beers[i] = beer_list
    
    
    return new_recs, recs[1], beers


In [584]:
np.random.seed(60)
ratings = {}
for i in recommender_df.columns:
    ratings[i] = np.random.uniform(low=3.5, high=5)

recs = data_manipulation(ratings)

In [585]:
recs[0]

,brewery,similar_beers,score
18,Stone Brewing Co.,7,0.406850
3,"Bell's Brewery, Inc.",5,0.399865
10,Founders Brewing Company,5,0.436333
17,Sierra Nevada Brewing Co.,3,0.417807
16,Russian River Brewing Company,3,0.427895
19,Three Floyds Brewing Co. & Brewpub,2,0.403027
9,Dogfish Head Brewery,2,0.419419
2,Bear Republic Brewing Co.,2,0.430955
0,Anchor Brewing Company,2,0.432422


In [580]:
recs[2]

{'Stone Brewing Co.': ['Stone IPA (India Pale Ale)',
  'Oaked Arrogant Bastard Ale',
  'Stone Smoked Porter'],
 "Bell's Brewery, Inc.": ["Bell's Kalamazoo Stout", "Bell's Oberon Ale"],
 'Boston Beer Company (Samuel Adams)': ['Samuel Adams Boston Lager'],
 'Three Floyds Brewing Co. & Brewpub': ['Alpha King Pale Ale']}

In [445]:
breweries = []
current_beer = ''
for beer in beer_checklist:
    for index, brewery in df[df['beer_name'] == beer[0]]['brewery_name'].items():
        if current_beer != beer[0]:
            current_beer = beer[0] 
            brewery_score = (brewery, beer[1], beer[0])
            breweries.append(brewery_score) 

In [446]:
breweries

[("Bell's Brewery, Inc.", 0.4610159776216624, "Bell's Expedition Stout"),
 ("Bell's Brewery, Inc.", 0.4783891468993994, "Bell's Hopslam Ale"),
 ("Bell's Brewery, Inc.", 0.4908797580696664, "Bell's Kalamazoo Stout"),
 ('Three Floyds Brewing Co. & Brewpub',
  0.4942236122391343,
  'Alpha King Pale Ale'),
 ('Founders Brewing Company', 0.5027620221233815, 'Founders Breakfast Stout'),
 ('Founders Brewing Company', 0.5061173218030857, "Founders Red's Rye PA"),
 ("Bell's Brewery, Inc.", 0.5074267084119577, "Bell's Oberon Ale"),
 ('Sierra Nevada Brewing Co.',
  0.5087896217582729,
  'Sierra Nevada Celebration Ale'),
 ('Stone Brewing Co.', 0.5107534990349647, 'Stone IPA (India Pale Ale)'),
 ('Founders Brewing Company', 0.5119857672626005, 'Founders Centennial IPA'),
 ('Three Floyds Brewing Co. & Brewpub', 0.5122005525837294, 'Dreadnaught IPA'),
 ('Stone Brewing Co.', 0.5128667093104494, 'Stone Ruination IPA'),
 ('Great Lakes Brewing Company',
  0.514209837006248,
  'Great Lakes Edmund Fitzgeral

In [438]:
breweries

[("Bell's Brewery, Inc.", 0.4610159776216624, "Bell's Expedition Stout"),
 ("Bell's Brewery, Inc.", 0.4783891468993994, "Bell's Hopslam Ale"),
 ("Bell's Brewery, Inc.", 0.4908797580696664, "Bell's Kalamazoo Stout"),
 ('Three Floyds Brewing Co. & Brewpub',
  0.4942236122391343,
  'Alpha King Pale Ale'),
 ('Founders Brewing Company', 0.5027620221233815, 'Founders Breakfast Stout'),
 ('Founders Brewing Company', 0.5061173218030857, "Founders Red's Rye PA"),
 ("Bell's Brewery, Inc.", 0.5074267084119577, "Bell's Oberon Ale"),
 ('Sierra Nevada Brewing Co.',
  0.5087896217582729,
  'Sierra Nevada Celebration Ale'),
 ('Stone Brewing Co.', 0.5107534990349647, 'Stone IPA (India Pale Ale)'),
 ('Founders Brewing Company', 0.5119857672626005, 'Founders Centennial IPA'),
 ('Stone Brewing Co.', 0.4364264441770118, 'Stone IPA (India Pale Ale)'),
 ('Stone Brewing Co.', 0.4570314364348086, 'Arrogant Bastard Ale'),
 ('Bear Republic Brewing Co.', 0.4747606537867224, 'Hop Rod Rye'),
 ('Dogfish Head Brewery

In [447]:
# sorting the breweries alphabetically to make it easier to loop through
current_brewery = ''
scores = []
brewery_dict = {}
beer_dict = {}
beers_dict = {}
breweries_sort = sorted(breweries)
for i in range(len(breweries_sort)):
    if breweries_sort[i][0] == current_brewery:
        current_beer = breweries_sort[i][2]
        beer_dict[current_beer] = breweries_sort[i][1]      
        scores.append(breweries_sort[i][1])
    else:
        if current_brewery:
            brewery_dict[current_brewery] = scores
            beers_dict[current_brewery] = beer_dict
            beer_dict = {}
        current_brewery = breweries_sort[i][0]
        current_beer = breweries_sort[i][2]

        beer_dict[current_beer] = breweries_sort[i][1]     
        scores = []
        scores.append(breweries_sort[i][1])

In [448]:
beers_dict

{'Bear Republic Brewing Co.': {'Hop Rod Rye': 0.5299445783171741,
  'Racer 5 India Pale Ale': 0.5386763715374788},
 "Bell's Brewery, Inc.": {"Bell's Expedition Stout": 0.4610159776216624,
  "Bell's Hopslam Ale": 0.4783891468993994,
  "Bell's Kalamazoo Stout": 0.4908797580696664,
  "Bell's Oberon Ale": 0.5074267084119577,
  'Two Hearted Ale': 0.5128667093104494,
  "Bell's Special Double Cream Stout": 0.5329256052957156},
 'Brooklyn Brewery': {'Brooklyn Black Chocolate Stout': 0.539124847425458},
 'Brouwerij St. Bernardus NV': {'St. Bernardus Abt 12': 0.5339438162028565},
 'Dogfish Head Brewery': {'90 Minute IPA': 0.5190909937654438,
  '60 Minute IPA': 0.5372972611187194},
 'Founders Brewing Company': {'Founders Breakfast Stout': 0.5027620221233815,
  "Founders Red's Rye PA": 0.5061173218030857,
  'Founders Centennial IPA': 0.5119857672626005},
 'Great Lakes Brewing Company': {'Great Lakes Edmund Fitzgerald Porter': 0.514209837006248},
 'North Coast Brewing Co.': {'Old Rasputin Russian I

In [441]:
beers_dict

{'Bear Republic Brewing Co.': {'Hop Rod Rye': 0.4747606537867224,
  'Racer 5 India Pale Ale': 0.5017039243600097},
 "Bell's Brewery, Inc.": {"Bell's Expedition Stout": 0.4610159776216624,
  "Bell's Hopslam Ale": 0.4783891468993994,
  "Bell's Kalamazoo Stout": 0.4908797580696664,
  "Bell's Oberon Ale": 0.5074267084119577},
 'Dogfish Head Brewery': {'90 Minute IPA': 0.4788566410450421},
 'Founders Brewing Company': {'Founders Breakfast Stout': 0.5027620221233815,
  "Founders Red's Rye PA": 0.5061173218030857,
  'Founders Centennial IPA': 0.5119857672626005},
 'North Coast Brewing Co.': {'Old Rasputin Russian Imperial Stout': 0.4973950938896031},
 'Sierra Nevada Brewing Co.': {'Sierra Nevada Celebration Ale': 0.5087896217582729},
 'Stone Brewing Co.': {'Stone IPA (India Pale Ale)': 0.5107534990349647,
  'Arrogant Bastard Ale': 0.4570314364348086,
  'Double Bastard Ale': 0.4791647659602123,
  'Stone Imperial Russian Stout': 0.4832634113289038,
  'Oaked Arrogant Bastard Ale': 0.493042769104

In [425]:
check[1]

[("Bell's Expedition Stout", 0.4610159776216624),
 ("Bell's Hopslam Ale", 0.4783891468993994),
 ("Bell's Kalamazoo Stout", 0.4908797580696664),
 ('Alpha King Pale Ale', 0.4942236122391343),
 ('Founders Breakfast Stout', 0.5027620221233815),
 ("Founders Red's Rye PA", 0.5061173218030857),
 ("Bell's Oberon Ale", 0.5074267084119577),
 ('Sierra Nevada Celebration Ale', 0.5087896217582729),
 ('Stone IPA (India Pale Ale)', 0.5107534990349647),
 ('Founders Centennial IPA', 0.5119857672626005),
 ('Dreadnaught IPA', 0.5122005525837294),
 ('Stone Ruination IPA', 0.5128667093104494),
 ('Great Lakes Edmund Fitzgerald Porter', 0.514209837006248),
 ('Sierra Nevada Bigfoot Barleywine Style Ale', 0.5157168659746445),
 ('90 Minute IPA', 0.5190909937654438),
 ('HopDevil Ale', 0.5271312075828197),
 ('Brooklyn Black Chocolate Stout', 0.5275642083819141),
 ('Hop Rod Rye', 0.5299445783171741),
 ('Storm King Stout', 0.5300206666099974),
 ('Old Rasputin Russian Imperial Stout', 0.5308792460977849),
 ("Bell's 

In [419]:
np.random.seed(60)
ratings = {}
for i in recommender_df.columns:
    ratings[i] = np.random.uniform(low=2, high=5)

recs = data_manipulation(ratings)

In [417]:
recs[0]

,brewery,similar_beers,score
6,Stone Brewing Co.,4,0.484651
1,"Bell's Brewery, Inc.",3,0.484428
2,Dogfish Head Brewery,1,0.478857
0,Bear Republic Brewing Co.,1,0.488232
4,North Coast Brewing Co.,1,0.497395


In [420]:
recs[0]

,brewery,similar_beers,score
3,"Bell's Brewery, Inc.",2,0.530179
25,Three Floyds Brewing Co. & Brewpub,1,0.536177


In [405]:
recs[2]

{'Stone Brewing Co.': ['Arrogant Bastard Ale',
  'Double Bastard Ale',
  'Stone Imperial Russian Stout',
  'Oaked Arrogant Bastard Ale'],
 "Bell's Brewery, Inc.": ["Bell's Expedition Stout",
  "Bell's Hopslam Ale",
  "Bell's Kalamazoo Stout"],
 'Three Floyds Brewing Co. & Brewpub': ['Alpha King Pale Ale']}

In [392]:
list(recs[0])

['brewery', 'similar_beers', 'score']

In [254]:
for i in a, b:
    print(i)
    for j in i:
        print(j)


[1, 2, 3, 4, 5]
1
2
3
4
5
['a', 'b', 'c', 4, 'f']
a
b
c
4
f


In [245]:
for brewery, beers in recs[1].items():
    if len(beers) >= 1:
        

{'Bear Republic Brewing Co.': ['Hop Rod Rye'],
 "Bell's Brewery, Inc.": ["Bell's Expedition Stout",
  "Bell's Hopslam Ale",
  "Bell's Kalamazoo Stout"],
 'Dogfish Head Brewery': ['90 Minute IPA'],
 'Founders Brewing Company': [],
 'North Coast Brewing Co.': ['Old Rasputin Russian Imperial Stout'],
 'Sierra Nevada Brewing Co.': [],
 'Stone Brewing Co.': ['Arrogant Bastard Ale',
  'Double Bastard Ale',
  'Stone Imperial Russian Stout',
  'Oaked Arrogant Bastard Ale']}

# Scraping ratebeer

In [2]:
# scraping for dc breweries
url = 'https://www.ratebeer.com/breweries/washington-dc/48/213/'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'lxml')

In [4]:
dc_breweries = []
for i in soup.find_all('div', {'class' : 'tab-pane active searchable'}):
    for j in i.find_all('table', {'id' : 'brewerTable'}):
        for k in j.find_all('td'):
            try:
                dc_breweries.append(k.find('a').text)
            except:
                pass

In [7]:
dc_breweries_spaces = []
for i in dc_breweries:
    dc_breweries_spaces.append(i.strip())

In [8]:
dc_breweries_spaces

['3 Stars Brewing Company',
 'Atlas Brew Works',
 'Bardo Brewing',
 'Bluejacket',
 'ChopHouse & Brewery D.C. (Craftworks)',
 'DC Brau Brewing Company',
 'Georgetown Trading Company',
 'Hellbender Brewing Company',
 'Red Bear Brewing Company',
 'Right Proper Brewing Company',
 'Sankofa Beer Company',
 'The Public Option',
 'The Pug',
 'Valor Brewpub']

In [30]:
df[df['brewery_name'].str.contains('Port City')]

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
459401,24791,Port City Brewing,2012-01-01 01:07:46,3.5,3.0,4.0,brentk56,American Pale Ale (APA),4.0,3.0,Essential Pale Ale,5.5,65927
459402,24791,Port City Brewing,2011-12-17 23:17:13,3.5,3.5,3.5,paulginva,American Pale Ale (APA),3.0,4.0,Essential Pale Ale,5.5,65927
459403,24791,Port City Brewing,2011-12-10 02:53:24,4.0,4.0,3.0,beer2day,American Pale Ale (APA),4.0,4.0,Essential Pale Ale,5.5,65927
459404,24791,Port City Brewing,2011-12-02 23:08:46,2.0,2.0,2.5,wowbagger,American Pale Ale (APA),2.5,3.0,Essential Pale Ale,5.5,65927
459405,24791,Port City Brewing,2011-11-02 02:19:32,3.5,3.0,3.5,LaCoursiere,American Pale Ale (APA),3.5,4.0,Essential Pale Ale,5.5,65927
459406,24791,Port City Brewing,2011-11-01 23:07:56,3.5,3.5,3.5,properhops,American Pale Ale (APA),3.0,4.0,Essential Pale Ale,5.5,65927
459407,24791,Port City Brewing,2011-09-30 20:41:14,5.0,5.0,4.5,Emerson1310,American Pale Ale (APA),4.5,5.0,Essential Pale Ale,5.5,65927
459408,24791,Port City Brewing,2011-08-22 23:00:17,4.0,4.0,4.0,Rhettroactive,American Pale Ale (APA),4.0,4.0,Essential Pale Ale,5.5,65927
459409,24791,Port City Brewing,2011-08-22 11:58:08,3.0,2.5,3.0,BeerFMAndy,American Pale Ale (APA),3.0,3.5,Essential Pale Ale,5.5,65927
459410,24791,Port City Brewing,2011-08-19 20:18:59,3.5,3.5,4.0,jayhawk73,American Pale Ale (APA),3.5,4.0,Essential Pale Ale,5.5,65927
